# **HW1: Regression** 
In *assignment 1*, you need to finish:

1.  Basic Part: Implement the regression model to predict the number of dengue cases


> *   Step 1: Split Data
> *   Step 2: Preprocess Data
> *   Step 3: Implement Regression
> *   Step 4: Make Prediction
> *   Step 5: Train Model and Generate Result

2.  Advanced Part: Implementing a regression model to predict the number of dengue cases in a different way than the basic part

# 1. Basic Part (60%)
In the first part, you need to implement the regression to predict the number of dengue cases

Please save the prediction result in a csv file **hw1_basic.csv**


## Import Packages

> Note: You **cannot** import any other package in the basic part

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import math
import random

## Global attributes
Define the global attributes

In [2]:
# Input file named as 'hw1_basic_input.csv'
input_dataroot = 'hw1_basic_input.csv'
output_dataroot = 'hw1_basic.csv'  # Output file will be named as 'hw1_basic.csv'

input_datalist = []  # Initial datalist, saved as numpy array
output_datalist = []  # Your prediction, should be 10 * 4 matrix and saved as numpy array
# The format of each row should be ['epiweek', 'CityA', 'CityB', 'CityC']

output_datalist = np.zeros((10,4)).astype(int)


You can add your own global attributes here


In [3]:
# Number of training data and validation data
train_num = 84
valid_num = 10

# Data set
X_train = np.zeros((train_num, 3), dtype=float)  
Y_train = np.zeros((train_num, 3), dtype=int)
X_valid = np.zeros((valid_num, 3), dtype=float)
Y_valid = np.zeros((valid_num, 3), dtype=int)

## Load the Input File
First, load the basic input file **hw1_basic_input.csv**

Input data would be stored in *input_datalist*

In [4]:
# Read input csv to datalist
with open(input_dataroot, newline='') as csvfile:
    input_datalist = np.array(list(csv.reader(csvfile))).reshape((105,7))
    
# Predict Set
X_Predict = np.array(input_datalist[95:105, 1:4]).astype(float)

## Implement the Regression Model

> Note: It is recommended to use the functions we defined, you can also define your own functions


### Step 1: Split Data
Split data in *input_datalist* into training dataset and validation dataset 



In [5]:
def SplitData(train_num, valid_num):
    # Process data list and permutation
    data_array = np.array(input_datalist[1:95, 1:])
    # np.random.shuffle(data_array)

    x_array = np.array(data_array[:, 0:3])
    y_array = np.array(data_array[:, 3:6])

    i = 0
    for data_list in x_array:
        j = 0
        for data in data_list:
            if(data == ''):
                # Solve undefine data
                x_array[i][j] = 0.0
            else:
                x_array[i][j] = float(data)
            j += 1
        i += 1

    # Split data
    global X_train, Y_train, X_valid, Y_valid
    X_train = np.array(x_array[0:train_num]).astype(float)
    Y_train = np.array(y_array[0:train_num]).astype(int)
    X_valid = np.array(x_array[train_num:train_num+valid_num]).astype(float)
    Y_valid = np.array(y_array[train_num:train_num+valid_num]).astype(int)

### Step 2: Preprocess Data
Handle the unreasonable data
> Hint: Outlier and missing data can be handled by removing the data or adding the values with the help of statistics  

In [6]:
def PreprocessData(data_list):
    num = len(data_list)
    mean = data_list.mean()
    std = data_list.std()
    z_score = np.array((data_list - mean) / std)
    rtn_list = data_list
    for i in range(0, num):
        if(z_score[i] < -1.5 or z_score[i] > 1.5):
            rtn_list[i] = mean
    return rtn_list

### Step 3: Implement Regression
> Hint: You can use Matrix Inversion, or Gradient Descent to finish this part

##### Linear Model
 $$f_{w,b}(x^{(i)}) = wx^{(i)} + b \tag{1}$$

##### Cost Function With Regularization
$$J(\mathbf{w},b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})^2  + \frac{\lambda}{2m}  \sum_{j=0}^{n-1} w_j^2 \tag{1}$$ 

##### Gradient Descent
$$\begin{align*} \text{repeat}&\text{ until convergence:} \; \lbrace \newline
\;  w &= w -  \alpha \frac{\partial J(w,b)}{\partial w} \tag{3}  \; \newline 
 b &= b -  \alpha \frac{\partial J(w,b)}{\partial b}  \newline \rbrace
\end{align*}$$

##### Gradient
$$
\begin{align}
\frac{\partial J(w,b)}{\partial w}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})x^{(i)} \tag{4}  \; \newline
  \frac{\partial J(w,b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)}) \newline
\end{align}
$$



In [7]:
# Cost Function
def compute_cost(x, y, w, b, Lambda):
    # Number of training examples
    m = x.shape[0]
    n = len(w)
    cost = 0.0
        
    for i in range(m):
        f_wb = np.dot(x[i], w) + b          #(n,)(n,) = scalar (see np.dot)
        cost += (f_wb - y[i]) ** 2           #scalar
    total_cost = float((1 / (2 * m)) * cost)   #scalar
    
    # Regularization
    reg_cost = 0
    for j in range(n):
        reg_cost += (w[j]**2)                      #scalar
    reg_cost = (Lambda / (2 * m)) * reg_cost
    
    rtn_cost = total_cost + reg_cost
    
    return rtn_cost

In [8]:
#Gradient
def compute_gradient(x, y, w, b,Lambda):
    # Number of training examples
    m,n = x.shape               #(number of examples, number of features)
    dj_dw = np.zeros((n,))
    dj_db = 0.
    
    for i in range(m):  
        err = (np.dot(x[i], w) + b) - y[i]
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err * x[i, j]
        dj_db = dj_db + err
    dj_dw = dj_dw / m 
    dj_db = dj_db / m
    
    for j in range(n):
        dj_dw[j] = dj_dw[j] + (Lambda / m) * w[j] 
        
    return dj_dw, dj_db
        

In [9]:
# Gradient Descent
def Regression(x, y, w_in, b_in, alpha, Lambda, num_iters, cost_function, gradient_function):
    # An array to store cost J and w's at each iteration primarily for graphing later
    J_history = []
    b = b_in
    w = w_in

    for i in range(num_iters):
        # Calculate the gradient and update the parameters using gradient_function
        dj_dw, dj_db = gradient_function(x, y, w, b, Lambda)

        # Update Parameters using w, b, alpha and gradient
        b = b - alpha * dj_db
        w = w - alpha * dj_dw

        # Save cost J at each iteration
        if i < 100000:      # prevent resource exhaustion
            J_history.append(cost_function(x, y, w, b, Lambda))
        # Print cost every at intervals 10 times or as many iterations if < 10
        if i % math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]:8.2f}")
            # print("iterations: ", i, "Cost: ", J_history[-1])
            
    return w, float(b), J_history  # return w b and J history for graphing


### Step 4: Make Prediction
Make prediction of testing dataset and store the value in *output_datalist*

In [10]:
def MakePrediction(x, w, b):
    m = x.shape[0]
    y_hat = np.zeros(m)
    for i in range(m):
    #     y_hat[i] = w[0]*x[i]**3 + w[1]*x[i]**2 + w[2]*x[i] + b
        y_hat[i] = int(np.dot(x[i], w) + b)

    # y_hat = np.array(np.dot(x, w) + b).astype(int)

    return y_hat

In [11]:
def MakeSinglePrediction(x_temp, x1, x2, x3, x4, x5, w, b):
    y_hat = int(x_temp*w[0] + x1*w[1] + x2*w[2] + x3*w[3] + x4*w[4] + x5*w[5] + b)
    
    return y_hat    

### Step 5: Train Model and Generate Result

> Notice: **Remember to output the coefficients of the model here**, otherwise 5 points would be deducted
* If your regression model is *2x^2 + 2x^1 + 1*, your output would be: 
```
2 2 1
```





### **ARMA Model**

In [12]:
def moving_average(beta, data_list):
    num = len(data_list)
    v_init = 0
    v = np.zeros(num)
    v[0] = beta * v_init + (1 - beta) * data_list[0]
    temp = v[0] / (1 - beta)
    v[0] = temp
    for i in range(1, num):
        v[i] = beta * v[i-1] + (1 - beta) * data_list[i]
    return v

In [13]:
def arma_model(x_list, y_list):
    m = y_list.shape[0]
    one_before = np.zeros(m).reshape(-1,1).astype(int)
    two_before = np.zeros(m).reshape(-1,1).astype(int)
    three_before = np.zeros(m).reshape(-1,1).astype(int)
    four_before = np.zeros(m).reshape(-1,1).astype(int)
    five_before = np.zeros(m).reshape(-1,1).astype(int)
    
    # One day before
    for i in range(1,m):
        one_before[i] = y_list[i-1]
        
    # Two day before
    for i in range(2,m):
        two_before[i] = y_list[i-2]
        
    # Three day before    
    for i in range(3,m):
        three_before[i] = y_list[i-3]
        
    # Four day before
    for i in range(4,m):
        four_before[i] = y_list[i-4]
        
    # Five day before
    for i in range(5,m):
        five_before[i] = y_list[i-5]
        
    rtn_array = np.concatenate([x_list, one_before, two_before, three_before, four_before, five_before], axis=1)

    return rtn_array        


### **MAPE**

In [14]:
# Evaluate MAPE
def evaluate_MAPE(n, y_hat, y):
    error_sum = 0
    for i in range(n):
        abs_error_i = abs(y[i] - y_hat[i])
        error_i = abs_error_i / y[i]
        error_sum += error_i
    error_sum = float(error_sum / n * 100)
    
    return error_sum

### **Preprocess Data Set**

In [15]:
# Split data

SplitData(train_num, valid_num)

# PreProcess Data
# PreprocessData(train_num, valid_num)
cityA_temp_Train = np.array(PreprocessData(X_train[:, 0])).reshape(-1,1)
cityB_temp_Train = np.array(PreprocessData(X_train[:, 1])).reshape(-1,1)
cityC_temp_Train = np.array(PreprocessData(X_train[:, 2])).reshape(-1,1)

cityA_X_Valid = np.array(X_valid[:, 0]).reshape(-1,1)
cityB_X_Valid = np.array(X_valid[:, 1]).reshape(-1,1)
cityC_X_Valid = np.array(X_valid[:, 2]).reshape(-1,1)

cityA_Y_Train = np.array(Y_train[:, 0]).reshape(-1,1)
cityB_Y_Train = np.array(Y_train[:, 1]).reshape(-1,1)
cityC_Y_Train = np.array(Y_train[:, 2]).reshape(-1,1)

cityA_Y_Valid = np.array(Y_valid[:, 0]).reshape(-1,1)
cityB_Y_Valid = np.array(Y_valid[:, 1]).reshape(-1,1)
cityC_Y_Valid = np.array(Y_valid[:, 2]).reshape(-1,1)

cityA_X_Train = arma_model(cityA_temp_Train, cityA_Y_Train)
cityB_X_Train = arma_model(cityB_temp_Train, cityB_Y_Train)
cityC_X_Train = arma_model(cityC_temp_Train, cityC_Y_Train)

cityA_X_All = arma_model(np.append(cityA_temp_Train, cityA_X_Valid).reshape(-1,1), np.append(cityA_Y_Train, cityA_Y_Valid).reshape(-1,1))
cityB_X_All = arma_model(np.append(cityB_temp_Train, cityB_X_Valid).reshape(-1,1), np.append(cityB_Y_Train, cityB_Y_Valid).reshape(-1,1))
cityC_X_All = arma_model(np.append(cityC_temp_Train, cityC_X_Valid).reshape(-1,1), np.append(cityC_Y_Train, cityC_Y_Valid).reshape(-1,1))

cityA_Y_All = np.append(cityA_Y_Train, cityA_Y_Valid).reshape(-1,1)
cityB_Y_All = np.append(cityB_Y_Train, cityB_Y_Valid).reshape(-1,1)
cityC_Y_All = np.append(cityC_Y_Train, cityC_Y_Valid).reshape(-1,1)


# arma_A_Train = np.array(compute_arma(beta_arma, cityA_Y_Train)).reshape(-1,1)
# arma_B_Train = np.array(compute_arma(beta_arma, cityB_Y_Train)).reshape(-1,1)
# arma_C_Train = np.array(compute_arma(beta_arma, cityC_Y_Train)).reshape(-1,1)


# cityA_X_Train = np.concatenate([cityA_X_Train, arma_A_Train], axis=1)
# cityB_X_Train = np.concatenate([cityB_X_Train, arma_B_Train], axis=1)
# cityC_X_Train = np.concatenate([cityC_X_Train, arma_C_Train], axis=1)


# arma_A_Valid = np.array(compute_arma(beta_arma, cityA_Y_Valid))
# arma_B_Valid = np.array(compute_arma(beta_arma, cityB_Y_Valid))
# arma_C_Valid = np.array(compute_arma(beta_arma, cityC_Y_Valid))

# print(arma_B_Train)
# plt.plot(arma_C_Train,'ro', cityC_Y_Train, 'bo')

# Visualizing data distribution
# plt.scatter(np.array(X_train[:,0]), np.array(Y_train[:,0]))
# plt.plot(Y_train[:,2])


### **Initial Parameters**

In [16]:
# w_init = np.zeros(3)
# w_init = np.zeros(6)
# w_init = 0
b_init = 0
v_init = 0
beta_arma = 0.9

# learning_rate = 3.0e-3
learning_rate_A = 8.5e-5
learning_rate_B = 2.0e-4
learning_rate_C = 3.0e-4

# Regularization lambda
lambda_A = 3.0e5
lambda_B = 3.5e4
lambda_C = 9.0e4

iterations = 5000

### **Model Training**

In [17]:
# w_init = np.zeros(cityA_X_Train.shape[1])  # n features

# Traning City A
print("Training CityA")
# w_final_A, b_final_A, J_hist_A = Regression(
#     cityA_X_Train, cityA_Y_Train, w_init, b_init, learning_rate_A, Lambda, iterations, compute_cost, compute_gradient)

w_init = np.zeros(cityA_X_All.shape[1])

w_final_A, b_final_A, J_hist_A = Regression(
    cityA_X_All, cityA_Y_All, w_init, b_init, learning_rate_A, lambda_A, iterations, compute_cost, compute_gradient
)

Training CityA
Iteration    0: Cost  2009.18
Iteration  500: Cost   598.85
Iteration 1000: Cost   594.00
Iteration 1500: Cost   589.27
Iteration 2000: Cost   584.67
Iteration 2500: Cost   580.20
Iteration 3000: Cost   575.84
Iteration 3500: Cost   571.60
Iteration 4000: Cost   567.48
Iteration 4500: Cost   563.46


In [18]:
# Traning City B
print("Training CityB")
# w_final_B, b_final_B, J_hist_B = Regression(
#     cityB_X_Train, cityB_Y_Train, w_init, b_init, learning_rate_B, Lambda, iterations, compute_cost, compute_gradient)

w_final_B, b_final_B, J_hist_B = Regression(
    cityB_X_All, cityB_Y_All, w_init, b_init, learning_rate_B, lambda_B, iterations, compute_cost, compute_gradient
)

Training CityB
Iteration    0: Cost   195.67
Iteration  500: Cost   123.96
Iteration 1000: Cost   123.03
Iteration 1500: Cost   122.13
Iteration 2000: Cost   121.26
Iteration 2500: Cost   120.41
Iteration 3000: Cost   119.59
Iteration 3500: Cost   118.79
Iteration 4000: Cost   118.02
Iteration 4500: Cost   117.27


In [19]:
print("Training CityC")
# w_final_C, b_final_C, J_hist_C = Regression(
#     cityC_X_Train, cityC_Y_Train, w_init, b_init, learning_rate_C, Lambda, iterations, compute_cost, compute_gradient)


w_final_C, b_final_C, J_hist_C = Regression(
    cityC_X_All, cityC_Y_All, w_init, b_init, learning_rate_C, lambda_C, iterations, compute_cost, compute_gradient
)

Training CityC
Iteration    0: Cost   476.79
Iteration  500: Cost   127.08
Iteration 1000: Cost   125.29
Iteration 1500: Cost   123.68
Iteration 2000: Cost   122.25
Iteration 2500: Cost   120.97
Iteration 3000: Cost   119.83
Iteration 3500: Cost   118.81
Iteration 4000: Cost   117.90
Iteration 4500: Cost   117.08


### **Best Cost**
- City A : 563.46
- City B : 117.27
- City C : 117.08

### **Training Result**

In [20]:
# Model Training Result

y_train_hat = MakePrediction(cityA_X_Train, w_final_A, b_final_A)
print("BiasA:", evaluate_MAPE(train_num, y_train_hat, cityA_Y_Train),"%")
# plt.plot(cityA_Y_Train, 'bo', y_train_hat, 'ro')

y_train_hat = MakePrediction(cityB_X_Train, w_final_B, b_final_B)
print("BiasB:", evaluate_MAPE(train_num, y_train_hat, cityB_Y_Train),"%")
# plt.plot(cityB_Y_Train, 'bo', y_train_hat, 'ro')

y_train_hat = MakePrediction(cityC_X_Train, w_final_C, b_final_C)
print("BiasC:", evaluate_MAPE(train_num, y_train_hat, cityC_Y_Train),"%")
# plt.plot(cityC_Y_Train, 'bo', y_train_hat, 'ro')

# # plot cost versus iteration
# fig, (ax1, ax2) = plt.subplots(1, 2, constrained_layout=True, figsize=(12,4))
# ax1.plot(J_hist[:100])
# ax2.plot(1000 + np.arange(len(J_hist[1000:])), J_hist[1000:])
# ax1.set_title("Cost vs. iteration(start)");  ax2.set_title("Cost vs. iteration (end)")
# ax1.set_ylabel('Cost')            ;  ax2.set_ylabel('Cost')
# ax1.set_xlabel('iteration step')  ;  ax2.set_xlabel('iteration step')

BiasA: 21.29184541051772 %
BiasB: 18.628407798112747 %
BiasC: 41.38277107273743 %


### **Best Bias**
- City A : 21%
- City B : 18%
- City C : 41%

### **Validation Result**

In [21]:
# City A

y_valid_hat = np.zeros(cityA_X_Valid.shape[0])
y_valid_hat[0] = MakeSinglePrediction(
    cityA_X_Valid[0],
    cityA_Y_Train[83],
    cityA_Y_Train[82],
    cityA_Y_Train[81],
    cityA_Y_Train[80],
    cityA_Y_Train[79],
    w_final_A,
    b_final_A)

y_valid_hat[1] = MakeSinglePrediction(
    cityA_X_Valid[1],
    y_valid_hat[0],
    cityA_Y_Train[83],
    cityA_Y_Train[82],
    cityA_Y_Train[81],
    cityA_Y_Train[80],
    w_final_A,
    b_final_A)

y_valid_hat[2] = MakeSinglePrediction(
    cityA_X_Valid[2],
    y_valid_hat[1],
    y_valid_hat[0],
    cityA_Y_Train[83],
    cityA_Y_Train[82],
    cityA_Y_Train[81],
    w_final_A,
    b_final_A)

y_valid_hat[3] = MakeSinglePrediction(
    cityA_X_Valid[3],
    y_valid_hat[2],
    y_valid_hat[1],
    y_valid_hat[0],
    cityA_Y_Train[83],
    cityA_Y_Train[82],
    w_final_A,
    b_final_A)

y_valid_hat[4] = MakeSinglePrediction(
    cityA_X_Valid[4],
    y_valid_hat[3],
    y_valid_hat[2],
    y_valid_hat[1],
    y_valid_hat[0],
    cityA_Y_Train[83],
    w_final_A,
    b_final_A)

for i in range(5, valid_num):
    y_valid_hat[i] = MakeSinglePrediction(
        cityA_X_Valid[i],
        y_valid_hat[i-1],
        y_valid_hat[i-2],
        y_valid_hat[i-3],
        y_valid_hat[i-4],
        y_valid_hat[i-5],
        w_final_A,
        b_final_A
    )

print(y_valid_hat)

# y_valid_hat = MakePrediction(cityA_X_Valid, w_final_A, b_final_A)
print("VarrianceA:", evaluate_MAPE(valid_num, y_valid_hat, cityA_Y_Valid), "%")
# plt.plot(cityA_Y_Valid, 'bo', y_valid_hat, 'ro')

[32. 31. 31. 30. 28. 27. 26. 26. 25. 25.]
VarrianceA: 7.377467471170621 %


In [22]:
# City B

y_valid_hat = np.zeros(cityB_X_Valid.shape[0])
y_valid_hat[0] = MakeSinglePrediction(
    cityB_X_Valid[0],
    cityB_Y_Train[83],
    cityB_Y_Train[82],
    cityB_Y_Train[81],
    cityB_Y_Train[80],
    cityB_Y_Train[79],
    w_final_B,
    b_final_B)

y_valid_hat[1] = MakeSinglePrediction(
    cityB_X_Valid[1],
    y_valid_hat[0],
    cityB_Y_Train[83],
    cityB_Y_Train[82],
    cityB_Y_Train[81],
    cityB_Y_Train[80],
    w_final_B,
    b_final_B)

y_valid_hat[2] = MakeSinglePrediction(
    cityB_X_Valid[2],
    y_valid_hat[1],
    y_valid_hat[0],
    cityB_Y_Train[83],
    cityB_Y_Train[82],
    cityB_Y_Train[81],
    w_final_B,
    b_final_B)

y_valid_hat[3] = MakeSinglePrediction(
    cityB_X_Valid[3],
    y_valid_hat[2],
    y_valid_hat[1],
    y_valid_hat[0],
    cityB_Y_Train[83],
    cityB_Y_Train[82],
    w_final_B,
    b_final_B)

y_valid_hat[4] = MakeSinglePrediction(
    cityB_X_Valid[4],
    y_valid_hat[3],
    y_valid_hat[2],
    y_valid_hat[1],
    y_valid_hat[0],
    cityB_Y_Train[83],
    w_final_B,
    b_final_B)

for i in range(5, valid_num):
    y_valid_hat[i] = MakeSinglePrediction(
        cityB_X_Valid[i],
        y_valid_hat[i-1],
        y_valid_hat[i-2],
        y_valid_hat[i-3],
        y_valid_hat[i-4],
        y_valid_hat[i-5],
        w_final_B,
        b_final_B
    )

print(y_valid_hat)

# y_valid_hat = MakePrediction(cityB_X_Valid, w_final_B, b_final_B)
print("VarrianceB:", evaluate_MAPE(valid_num, y_valid_hat, cityB_Y_Valid), "%")
# plt.plot(cityB_Y_Valid, 'bo', y_valid_hat, 'ro')

[22. 21. 22. 22. 22. 22. 22. 22. 22. 22.]
VarrianceB: 18.32737520128824 %


In [23]:
#City C

y_valid_hat = np.zeros(cityC_X_Valid.shape[0])
y_valid_hat[0] = MakeSinglePrediction(
    cityC_X_Valid[0],
    cityC_Y_Train[83],
    cityC_Y_Train[82],
    cityC_Y_Train[81],
    cityC_Y_Train[80],
    cityC_Y_Train[79],
    w_final_C,
    b_final_C)

y_valid_hat[1] = MakeSinglePrediction(
    cityC_X_Valid[1],
    y_valid_hat[0],
    cityC_Y_Train[83],
    cityC_Y_Train[82],
    cityC_Y_Train[81],
    cityC_Y_Train[80],
    w_final_C,
    b_final_C)

y_valid_hat[2] = MakeSinglePrediction(
    cityC_X_Valid[2],
    y_valid_hat[1],
    y_valid_hat[0],
    cityC_Y_Train[83],
    cityC_Y_Train[82],
    cityC_Y_Train[81],
    w_final_C,
    b_final_C)

y_valid_hat[3] = MakeSinglePrediction(
    cityC_X_Valid[3],
    y_valid_hat[2],
    y_valid_hat[1],
    y_valid_hat[0],
    cityC_Y_Train[83],
    cityC_Y_Train[82],
    w_final_C,
    b_final_C)

y_valid_hat[4] = MakeSinglePrediction(
    cityC_X_Valid[4],
    y_valid_hat[3],
    y_valid_hat[2],
    y_valid_hat[1],
    y_valid_hat[0],
    cityC_Y_Train[83],
    w_final_C,
    b_final_C)

for i in range(5, valid_num):
    y_valid_hat[i] = MakeSinglePrediction(
        cityC_X_Valid[i],
        y_valid_hat[i-1],
        y_valid_hat[i-2],
        y_valid_hat[i-3],
        y_valid_hat[i-4],
        y_valid_hat[i-5],
        w_final_C,
        b_final_C
    )

print(y_valid_hat)

# y_valid_hat = MakePrediction(cityC_X_Valid, w_final_C, b_final_C)
print("VarrianceC:", evaluate_MAPE(valid_num, y_valid_hat, cityC_Y_Valid), "%")
# plt.plot(cityC_Y_Valid, 'bo', y_valid_hat, 'ro')

[46. 43. 42. 40. 38. 35. 33. 32. 30. 29.]
VarrianceC: 9.84737861142267 %


### **Best Varriance**
- City A : 7%
- City B : 18%
- City C : 9%
 
 

### **Prediction**

In [24]:
# City A
cityA_Predict = X_Predict[:,0]
y_hat_A = np.zeros(cityA_Predict.shape[0])

y_hat_A[0] = MakeSinglePrediction(
    cityA_Predict[0],
    cityA_Y_Valid[9],
    cityA_Y_Valid[8],
    cityA_Y_Valid[7],
    cityA_Y_Valid[6],
    cityA_Y_Valid[5],
    w_final_A,
    b_final_A)

y_hat_A[1] = MakeSinglePrediction(
    cityA_Predict[1],
    y_hat_A[0],
    cityA_Y_Valid[9],
    cityA_Y_Valid[8],
    cityA_Y_Valid[7],
    cityA_Y_Valid[6],
    w_final_A,
    b_final_A)

y_hat_A[2] = MakeSinglePrediction(
    cityA_Predict[2],
    y_hat_A[1],
    y_hat_A[0],
    cityA_Y_Valid[9],
    cityA_Y_Valid[8],
    cityA_Y_Valid[7],
    w_final_A,
    b_final_A)

y_hat_A[3] = MakeSinglePrediction(
    cityA_Predict[3],
    y_hat_A[2],
    y_hat_A[1],
    y_hat_A[0],
    cityA_Y_Valid[9],
    cityA_Y_Valid[8],
    w_final_A,
    b_final_A)

y_hat_A[4] = MakeSinglePrediction(
    cityA_Predict[4],
    y_hat_A[3],
    y_hat_A[2],
    y_hat_A[1],
    y_hat_A[0],
    cityA_Y_Valid[9],
    w_final_A,
    b_final_A)

for i in range(5, 10):
    y_hat_A[i] = MakeSinglePrediction(
        cityA_Predict[i],
        y_hat_A[i-1],
        y_hat_A[i-2],
        y_hat_A[i-3],
        y_hat_A[i-4],
        y_hat_A[i-5],
        w_final_A,
        b_final_A
    )

print(y_hat_A)

[24. 24. 23. 23. 23. 22. 22. 22. 22. 22.]


In [25]:
# City B
cityB_Predict = X_Predict[:,1]
y_hat_B = np.zeros(cityB_Predict.shape[0])

y_hat_B[0] = MakeSinglePrediction(
    cityB_Predict[0],
    cityB_Y_Valid[9],
    cityB_Y_Valid[8],
    cityB_Y_Valid[7],
    cityB_Y_Valid[6],
    cityB_Y_Valid[5],
    w_final_B,
    b_final_B)

y_hat_B[1] = MakeSinglePrediction(
    cityB_Predict[1],
    y_hat_B[0],
    cityB_Y_Valid[9],
    cityB_Y_Valid[8],
    cityB_Y_Valid[7],
    cityB_Y_Valid[6],
    w_final_B,
    b_final_B)

y_hat_B[2] = MakeSinglePrediction(
    cityB_Predict[2],
    y_hat_B[1],
    y_hat_B[0],
    cityB_Y_Valid[9],
    cityB_Y_Valid[8],
    cityB_Y_Valid[7],
    w_final_B,
    b_final_B)

y_hat_B[3] = MakeSinglePrediction(
    cityB_Predict[3],
    y_hat_B[2],
    y_hat_B[1],
    y_hat_B[0],
    cityB_Y_Valid[9],
    cityB_Y_Valid[8],
    w_final_B,
    b_final_B)

y_hat_B[4] = MakeSinglePrediction(
    cityB_Predict[4],
    y_hat_B[3],
    y_hat_B[2],
    y_hat_B[1],
    y_hat_B[0],
    cityB_Y_Valid[9],
    w_final_B,
    b_final_B)

for i in range(5, 10):
    y_hat_B[i] = MakeSinglePrediction(
        cityB_Predict[i],
        y_hat_B[i-1],
        y_hat_B[i-2],
        y_hat_B[i-3],
        y_hat_B[i-4],
        y_hat_B[i-5],
        w_final_B,
        b_final_B
    )

print(y_hat_B)

# y_hat_B = MakePrediction(cityB_Predict, w_final_B, b_final_B)
# print("VarrianceB:", evaluate_MAPE(valid_num, y_hat_B, cityB_Y_Valid), "%")
# plt.plot(cityB_Y_Valid, 'bo', y_hat_B, 'ro')

[23. 23. 23. 23. 23. 23. 23. 22. 22. 22.]


In [26]:
# City C
cityC_Predict = X_Predict[:,1]
y_hat_C = np.zeros(cityC_Predict.shape[0])

y_hat_C[0] = MakeSinglePrediction(
    cityC_Predict[0],
    cityC_Y_Valid[9],
    cityC_Y_Valid[8],
    cityC_Y_Valid[7],
    cityC_Y_Valid[6],
    cityC_Y_Valid[5],
    w_final_C,
    b_final_C)

y_hat_C[1] = MakeSinglePrediction(
    cityC_Predict[1],
    y_hat_C[0],
    cityC_Y_Valid[9],
    cityC_Y_Valid[8],
    cityC_Y_Valid[7],
    cityC_Y_Valid[6],
    w_final_C,
    b_final_C)

y_hat_C[2] = MakeSinglePrediction(
    cityC_Predict[2],
    y_hat_C[1],
    y_hat_C[0],
    cityC_Y_Valid[9],
    cityC_Y_Valid[8],
    cityC_Y_Valid[7],
    w_final_C,
    b_final_C)

y_hat_C[3] = MakeSinglePrediction(
    cityC_Predict[3],
    y_hat_C[2],
    y_hat_C[1],
    y_hat_C[0],
    cityC_Y_Valid[9],
    cityC_Y_Valid[8],
    w_final_C,
    b_final_C)

y_hat_C[4] = MakeSinglePrediction(
    cityC_Predict[4],
    y_hat_C[3],
    y_hat_C[2],
    y_hat_C[1],
    y_hat_C[0],
    cityC_Y_Valid[9],
    w_final_C,
    b_final_C)

for i in range(5, 10):
    y_hat_C[i] = MakeSinglePrediction(
        cityC_Predict[i],
        y_hat_C[i-1],
        y_hat_C[i-2],
        y_hat_C[i-3],
        y_hat_C[i-4],
        y_hat_C[i-5],
        w_final_C,
        b_final_C
    )

print(y_hat_C)

# y_hat_C = MakePrediction(cityC_Predict, w_final_C, b_final_C)
# print("VarrianceC:", evaluate_MAPE(valid_num, y_hat_C, cityC_Y_Valid), "%")
# plt.plot(cityC_Y_Valid, 'bo', y_hat_C, 'ro')

[30. 29. 28. 28. 27. 25. 24. 23. 23. 22.]


## Write the Output File
Write the prediction to output csv
> Format: 'epiweek', 'CityA', 'CityB', 'CityC'

In [27]:
# Output format
week = 202143
for i in range(0, 10):
    output_datalist[i][0] = week + i
    output_datalist[i][1] = y_hat_A[i]
    output_datalist[i][2] = y_hat_B[i]
    output_datalist[i][3] = y_hat_C[i]

with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    for row in output_datalist:
        writer.writerow(row)


# 2. Advanced Part (25%)
In the second part, you need to implement the regression in a different way than the basic part to help your predictions for the number of dengue cases

We provide you with two files **hw1_advanced_input1.csv** and **hw1_advanced_input2.csv** that can help you in this part

Please save the prediction result in a csv file **hw1_advanced.csv** 


### **Open File**

In [28]:
input_1_dataroot = 'hw1_advanced_input1.csv'
input_2_dataroot = 'hw1_advanced_input2.csv'
output_dataroot = 'hw1_advanced.csv'

input_1_datalist = []
input_2_datalist = []
output_datalist = np.zeros((10,4)).astype(int)

# Read input csv to datalist
with open(input_1_dataroot, newline='') as csvfile:
    input_1_datalist = np.array(np.array(list(csv.reader(csvfile)))[1:, 1:]).astype(float)

with open(input_2_dataroot, newline='') as csvfile:
    input_2_datalist = np.array(np.array(list(csv.reader(csvfile)))[1:, 1:]).astype(float)

### **Preprocess Data Set**

In [29]:
cityA_x2_Train = np.array(input_1_datalist[0:train_num,0]).reshape(-1,1)
cityB_x2_Train = np.array(input_1_datalist[0:train_num,1]).reshape(-1,1)
cityC_x2_Train = np.array(input_1_datalist[0:train_num,2]).reshape(-1,1)

cityA_x2_Valid = np.array(input_1_datalist[train_num:train_num+valid_num, 0]).reshape(-1,1)
cityB_x2_Valid = np.array(input_1_datalist[train_num:train_num+valid_num, 1]).reshape(-1,1)
cityC_x2_Valid = np.array(input_1_datalist[train_num:train_num+valid_num, 2]).reshape(-1,1)

cityA_x2_Predict = np.array(input_1_datalist[train_num+valid_num:, 0]).reshape(-1,1)
cityB_x2_Predict = np.array(input_1_datalist[train_num+valid_num:, 1]).reshape(-1,1)
cityC_x2_Predict = np.array(input_1_datalist[train_num+valid_num:, 2]).reshape(-1,1)

cityA_feature = np.array(input_2_datalist[0,:])
cityB_feature = np.array(input_2_datalist[1,:])
cityC_feature = np.array(input_2_datalist[2,:])

cityA_feature_Train = np.tile(cityA_feature, (train_num, 1))
cityB_feature_Train = np.tile(cityB_feature, (train_num, 1))
cityC_feature_Train = np.tile(cityC_feature, (train_num, 1))

cityA_feature_Valid = np.tile(cityA_feature, (valid_num, 1))
cityB_feature_Valid = np.tile(cityB_feature, (valid_num, 1))
cityC_feature_Valid = np.tile(cityC_feature, (valid_num, 1))

cityA_X2_Train = np.concatenate(
    [np.concatenate([cityA_X_Train, cityA_x2_Train], axis = 1), cityA_feature_Train], axis=1)
cityB_X2_Train = np.concatenate(
    [np.concatenate([cityB_X_Train, cityB_x2_Train], axis = 1), cityB_feature_Train], axis=1)
cityC_X2_Train = np.concatenate(
    [np.concatenate([cityC_X_Train, cityC_x2_Train], axis = 1), cityC_feature_Train], axis=1)

# cityA_X2_Train = np.concatenate([cityA_X_Train, cityA_x2_Train], axis = 1)
# cityB_X2_Train = np.concatenate([cityB_X_Train, cityB_x2_Train], axis = 1)
# cityC_X2_Train = np.concatenate([cityC_X_Train, cityC_x2_Train], axis = 1)

cityA_X2_All = np.concatenate([cityA_X_All, np.append(cityA_x2_Train, cityA_x2_Valid).reshape(-1,1)], axis = 1)
cityB_X2_All = np.concatenate([cityB_X_All, np.append(cityB_x2_Train, cityB_x2_Valid).reshape(-1,1)], axis = 1)
cityC_X2_All = np.concatenate([cityC_X_All, np.append(cityC_x2_Train, cityC_x2_Valid).reshape(-1,1)], axis = 1)

cityA_X2_Valid = np.concatenate([np.concatenate([cityA_X_Valid, cityA_x2_Valid], axis = 1), cityA_feature_Valid], axis=1)
cityB_X2_Valid = np.concatenate([np.concatenate([cityB_X_Valid, cityB_x2_Valid], axis = 1), cityB_feature_Valid], axis=1)
cityC_X2_Valid = np.concatenate([np.concatenate([cityC_X_Valid, cityC_x2_Valid], axis = 1), cityC_feature_Valid], axis=1)

### **Model Training**

In [30]:
# w_init = np.zeros(cityA_X2_Train.shape[1])
w_init = np.zeros(cityA_X2_All.shape[1])
b_init = 0
# alpha_A = 9.0e-5
# alpha_B = 2.0e-4
# alpha_C = 3.0e-4
# Lambda = 9.5e4

alpha_A = 7.5e-5
alpha_B = 2.8e-4
alpha_C = 3.0e-4

lambda_A = 3.0e5
lambda_B = 3.0e4
lambda_C = 8.0e4

iterations = 5000

In [31]:
print("Training CityA")
# w_final_A2, b_final_A2, J_hist_A2 = Regression(
#     cityA_X2_Train, cityA_Y_Train, w_init, b_init, alpha_A, lambda_A, iterations, compute_cost, compute_gradient)

w_final_A2, b_final_A2, J_hist_A2 = Regression(
    cityA_X2_All, cityA_Y_All, w_init, b_init, alpha_A, lambda_A, iterations, compute_cost, compute_gradient)

Training CityA
Iteration    0: Cost  1434.39
Iteration  500: Cost   599.00
Iteration 1000: Cost   594.71
Iteration 1500: Cost   590.53
Iteration 2000: Cost   586.45
Iteration 2500: Cost   582.46
Iteration 3000: Cost   578.57
Iteration 3500: Cost   574.77
Iteration 4000: Cost   571.05
Iteration 4500: Cost   567.43


In [32]:
print("Training CityB")
# w_final_B2, b_final_B2, J_hist_B2 = Regression(
#     cityB_X2_Train, cityB_Y_Train, w_init, b_init, alpha_B, lambda_B, iterations, compute_cost, compute_gradient)

w_final_B2, b_final_B2, J_hist_B2 = Regression(
    cityB_X2_All, cityB_Y_All, w_init, b_init, alpha_B, lambda_B, iterations, compute_cost, compute_gradient)


Training CityB
Iteration    0: Cost   514.64
Iteration  500: Cost   117.98
Iteration 1000: Cost   116.94
Iteration 1500: Cost   115.94
Iteration 2000: Cost   114.97
Iteration 2500: Cost   114.05
Iteration 3000: Cost   113.16
Iteration 3500: Cost   112.31
Iteration 4000: Cost   111.48
Iteration 4500: Cost   110.70


In [33]:
print("Training CityC")
# w_final_C2, b_final_C2, J_hist_C2 = Regression(
#     cityC_X2_Train, cityC_Y_Train, w_init, b_init, alpha_C, lambda_C, iterations, compute_cost, compute_gradient)

w_final_C2, b_final_C2, J_hist_C2 = Regression(
    cityC_X2_All, cityC_Y_All, w_init, b_init, alpha_C, lambda_C, iterations, compute_cost, compute_gradient)

Training CityC
Iteration    0: Cost   469.48
Iteration  500: Cost   118.89
Iteration 1000: Cost   117.51
Iteration 1500: Cost   116.28
Iteration 2000: Cost   115.17
Iteration 2500: Cost   114.18
Iteration 3000: Cost   113.28
Iteration 3500: Cost   112.47
Iteration 4000: Cost   111.75
Iteration 4500: Cost   111.09


### **Training Result**

In [34]:
# Model Training Result

y_train_hat = MakePrediction(cityA_X2_All, w_final_A2, b_final_A2)
print("BiasA:", evaluate_MAPE(train_num, y_train_hat, cityA_Y_Train),"%")
# plt.plot(cityA_Y_Train, 'bo', y_train_hat, 'ro')

y_train_hat = MakePrediction(cityB_X2_All, w_final_B2, b_final_B2)
print("BiasB:", evaluate_MAPE(train_num, y_train_hat, cityB_Y_Train),"%")
# plt.plot(cityB_Y_Train, 'bo', y_train_hat, 'ro')

y_train_hat = MakePrediction(cityC_X2_All, w_final_C2, b_final_C2)
print("BiasC:", evaluate_MAPE(train_num, y_train_hat, cityC_Y_Train),"%")
# plt.plot(cityC_Y_Train, 'bo', y_train_hat, 'ro')

BiasA: 21.27787878280024 %
BiasB: 18.760760879647993 %
BiasC: 41.49614659585961 %


### **Make Prediction**

In [35]:
def MakeAdvancePrediction(x_temp, x1, x2, x3, x4, x5, x_pre, w, b):
    y_hat = int(x_temp*w[0] + x1*w[1] + x2*w[2] + x3*w[3] + x4*w[4] + x5*w[5]+ x_pre*w[6] + b)
    
    return y_hat

### **Validation Result**

In [36]:
# City A

y_valid_hat = np.zeros(cityA_X_Valid.shape[0])
y_valid_hat[0] = MakeAdvancePrediction(
    cityA_X_Valid[0],
    cityA_Y_Train[83],
    cityA_Y_Train[82],
    cityA_Y_Train[81],
    cityA_Y_Train[80],
    cityA_Y_Train[79],
    cityA_x2_Valid[0],
    w_final_A2,
    b_final_A2)

y_valid_hat[1] = MakeAdvancePrediction(
    cityA_X_Valid[1],
    y_valid_hat[0],
    cityA_Y_Train[83],
    cityA_Y_Train[82],
    cityA_Y_Train[81],
    cityA_Y_Train[80],
    cityA_x2_Valid[1],
    w_final_A2,
    b_final_A2)

y_valid_hat[2] = MakeAdvancePrediction(
    cityA_X_Valid[2],
    y_valid_hat[1],
    y_valid_hat[0],
    cityA_Y_Train[83],
    cityA_Y_Train[82],
    cityA_Y_Train[81],
    cityA_x2_Valid[2],
    w_final_A2,
    b_final_A2)

y_valid_hat[3] = MakeAdvancePrediction(
    cityA_X_Valid[3],
    y_valid_hat[2],
    y_valid_hat[1],
    y_valid_hat[0],
    cityA_Y_Train[83],
    cityA_Y_Train[82],
    cityA_x2_Valid[3],
    w_final_A2,
    b_final_A2)

y_valid_hat[4] = MakeAdvancePrediction(
    cityA_X_Valid[4],
    y_valid_hat[3],
    y_valid_hat[2],
    y_valid_hat[1],
    y_valid_hat[0],
    cityA_Y_Train[83],
    cityA_x2_Valid[4],
    w_final_A2,
    b_final_A2)

for i in range(5, valid_num):
    y_valid_hat[i] = MakeAdvancePrediction(
        cityA_X_Valid[i],
        y_valid_hat[i-1],
        y_valid_hat[i-2],
        y_valid_hat[i-3],
        y_valid_hat[i-4],
        y_valid_hat[i-5],
        cityA_x2_Valid[i],
        w_final_A2,
        b_final_A2
    )

print(y_valid_hat)

# y_valid_hat = MakePrediction(cityA_X_Valid, w_final_A, b_final_A)
print("VarrianceA:", evaluate_MAPE(valid_num, y_valid_hat, cityA_Y_Valid), "%")
# plt.plot(cityA_Y_Valid, 'bo', y_valid_hat, 'ro')

[32. 31. 30. 29. 28. 27. 26. 25. 25. 24.]
VarrianceA: 7.948532964816758 %


In [37]:
# City B

y_valid_hat = np.zeros(cityB_X_Valid.shape[0])
y_valid_hat[0] = MakeAdvancePrediction(
    cityB_X_Valid[0],
    cityB_Y_Train[83],
    cityB_Y_Train[82],
    cityB_Y_Train[81],
    cityB_Y_Train[80],
    cityB_Y_Train[79],
    cityB_x2_Train[0],
    w_final_B2,
    b_final_B2)

y_valid_hat[1] = MakeAdvancePrediction(
    cityB_X_Valid[1],
    y_valid_hat[0],
    cityB_Y_Train[83],
    cityB_Y_Train[82],
    cityB_Y_Train[81],
    cityB_Y_Train[80],
    cityB_x2_Train[1],
    w_final_B2,
    b_final_B2)

y_valid_hat[2] = MakeAdvancePrediction(
    cityB_X_Valid[2],
    y_valid_hat[1],
    y_valid_hat[0],
    cityB_Y_Train[83],
    cityB_Y_Train[82],
    cityB_Y_Train[81],
    cityB_x2_Train[2],
    w_final_B2,
    b_final_B2)

y_valid_hat[3] = MakeAdvancePrediction(
    cityB_X_Valid[3],
    y_valid_hat[2],
    y_valid_hat[1],
    y_valid_hat[0],
    cityB_Y_Train[83],
    cityB_Y_Train[82],
    cityB_x2_Train[3],
    w_final_B2,
    b_final_B2)

y_valid_hat[4] = MakeAdvancePrediction(
    cityB_X_Valid[4],
    y_valid_hat[3],
    y_valid_hat[2],
    y_valid_hat[1],
    y_valid_hat[0],
    cityB_Y_Train[83],
    cityB_x2_Train[4],
    w_final_B2,
    b_final_B2)

for i in range(5, valid_num):
    y_valid_hat[i] = MakeAdvancePrediction(
        cityB_X_Valid[i],
        y_valid_hat[i-1],
        y_valid_hat[i-2],
        y_valid_hat[i-3],
        y_valid_hat[i-4],
        y_valid_hat[i-5],
        cityB_x2_Train[i],
        w_final_B2,
        b_final_B2
    )

print(y_valid_hat)

# y_valid_hat = MakePrediction(cityB_X_Valid, w_final_B2, b_final_B2)
print("VarrianceB:", evaluate_MAPE(valid_num, y_valid_hat, cityB_Y_Valid), "%")
# plt.plot(cityB_Y_Valid, 'bo', y_valid_hat, 'ro')

[22. 22. 22. 23. 23. 23. 23. 23. 23. 23.]
VarrianceB: 18.047979797979796 %


In [38]:
# City C

y_valid_hat = np.zeros(cityC_X_Valid.shape[0])
y_valid_hat[0] = MakeAdvancePrediction(
    cityC_X_Valid[0],
    cityC_Y_Train[83],
    cityC_Y_Train[82],
    cityC_Y_Train[81],
    cityC_Y_Train[80],
    cityC_Y_Train[79],
    cityC_x2_Train[0],
    w_final_C2,
    b_final_C2)

y_valid_hat[1] = MakeAdvancePrediction(
    cityC_X_Valid[1],
    y_valid_hat[0],
    cityC_Y_Train[83],
    cityC_Y_Train[82],
    cityC_Y_Train[81],
    cityC_Y_Train[80],
    cityC_x2_Train[1],
    w_final_C2,
    b_final_C2)

y_valid_hat[2] = MakeAdvancePrediction(
    cityC_X_Valid[2],
    y_valid_hat[1],
    y_valid_hat[0],
    cityC_Y_Train[83],
    cityC_Y_Train[82],
    cityC_Y_Train[81],
    cityC_x2_Train[2],
    w_final_C2,
    b_final_C2)

y_valid_hat[3] = MakeAdvancePrediction(
    cityC_X_Valid[3],
    y_valid_hat[2],
    y_valid_hat[1],
    y_valid_hat[0],
    cityC_Y_Train[83],
    cityC_Y_Train[82],
    cityC_x2_Train[3],
    w_final_C2,
    b_final_C2)

y_valid_hat[4] = MakeAdvancePrediction(
    cityC_X_Valid[4],
    y_valid_hat[3],
    y_valid_hat[2],
    y_valid_hat[1],
    y_valid_hat[0],
    cityC_Y_Train[83],
    cityC_x2_Train[4],
    w_final_C2,
    b_final_C2)

for i in range(5, valid_num):
    y_valid_hat[i] = MakeAdvancePrediction(
        cityC_X_Valid[i],
        y_valid_hat[i-1],
        y_valid_hat[i-2],
        y_valid_hat[i-3],
        y_valid_hat[i-4],
        y_valid_hat[i-5],
        cityC_x2_Train[i],
        w_final_C2,
        b_final_C2
    )

print(y_valid_hat)

# y_valid_hat = MakePrediction(cityC_X_Valid, w_final_C2, b_final_C2)
print("VarrianceC:", evaluate_MAPE(valid_num, y_valid_hat, cityC_Y_Valid), "%")
# plt.plot(cityC_Y_Valid, 'bo', y_valid_hat, 'ro')

[46. 44. 42. 40. 38. 35. 33. 32. 30. 29.]
VarrianceC: 9.620105884149947 %


### **Prediction**

In [39]:
# City A
cityA_Predict = X_Predict[:,0]
y_hat_A = np.zeros(cityA_Predict.shape[0])

y_hat_A[0] = MakeAdvancePrediction(
    cityA_Predict[0],
    cityA_Y_Valid[9],
    cityA_Y_Valid[8],
    cityA_Y_Valid[7],
    cityA_Y_Valid[6],
    cityA_Y_Valid[5],
    cityA_x2_Predict[0],
    w_final_A2,
    b_final_A2)

y_hat_A[1] = MakeAdvancePrediction(
    cityA_Predict[1],
    y_hat_A[0],
    cityA_Y_Valid[9],
    cityA_Y_Valid[8],
    cityA_Y_Valid[7],
    cityA_Y_Valid[6],
    cityA_x2_Predict[1],
    w_final_A2,
    b_final_A2)

y_hat_A[2] = MakeAdvancePrediction(
    cityA_Predict[2],
    y_hat_A[1],
    y_hat_A[0],
    cityA_Y_Valid[9],
    cityA_Y_Valid[8],
    cityA_Y_Valid[7],
    cityA_x2_Predict[2],
    w_final_A2,
    b_final_A2)

y_hat_A[3] = MakeAdvancePrediction(
    cityA_Predict[3],
    y_hat_A[2],
    y_hat_A[1],
    y_hat_A[0],
    cityA_Y_Valid[9],
    cityA_Y_Valid[8],
    cityA_x2_Predict[3],
    w_final_A2,
    b_final_A2)

y_hat_A[4] = MakeAdvancePrediction(
    cityA_Predict[4],
    y_hat_A[3],
    y_hat_A[2],
    y_hat_A[1],
    y_hat_A[0],
    cityA_Y_Valid[9],
    cityA_x2_Predict[4],
    w_final_A2,
    b_final_A2)

for i in range(5, 10):
    y_hat_A[i] = MakeAdvancePrediction(
        cityA_Predict[i],
        y_hat_A[i-1],
        y_hat_A[i-2],
        y_hat_A[i-3],
        y_hat_A[i-4],
        y_hat_A[i-5],
        cityA_x2_Predict[i],
        w_final_A2,
        b_final_A2
    )

print(y_hat_A)

[24. 23. 23. 23. 22. 22. 21. 21. 21. 21.]


In [40]:
# City B
cityB_Predict = X_Predict[:,1]
y_hat_B = np.zeros(cityB_Predict.shape[0])

y_hat_B[0] = MakeAdvancePrediction(
    cityB_Predict[0],
    cityB_Y_Valid[9],
    cityB_Y_Valid[8],
    cityB_Y_Valid[7],
    cityB_Y_Valid[6],
    cityB_Y_Valid[5],
    cityB_x2_Predict[0],
    w_final_B2,
    b_final_B2)

y_hat_B[1] = MakeAdvancePrediction(
    cityB_Predict[1],
    y_hat_B[0],
    cityB_Y_Valid[9],
    cityB_Y_Valid[8],
    cityB_Y_Valid[7],
    cityB_Y_Valid[6],
    cityB_x2_Predict[1],
    w_final_B2,
    b_final_B2)

y_hat_B[2] = MakeAdvancePrediction(
    cityB_Predict[2],
    y_hat_B[1],
    y_hat_B[0],
    cityB_Y_Valid[9],
    cityB_Y_Valid[8],
    cityB_Y_Valid[7],
    cityB_x2_Predict[2],
    w_final_B2,
    b_final_B2)

y_hat_B[3] = MakeAdvancePrediction(
    cityB_Predict[3],
    y_hat_B[2],
    y_hat_B[1],
    y_hat_B[0],
    cityB_Y_Valid[9],
    cityB_Y_Valid[8],
    cityB_x2_Predict[3],
    w_final_B2,
    b_final_B2)

y_hat_B[4] = MakeAdvancePrediction(
    cityB_Predict[4],
    y_hat_B[3],
    y_hat_B[2],
    y_hat_B[1],
    y_hat_B[0],
    cityB_Y_Valid[9],
    cityB_x2_Predict[4],
    w_final_B2,
    b_final_B2)

for i in range(5, 10):
    y_hat_B[i] = MakeAdvancePrediction(
        cityB_Predict[i],
        y_hat_B[i-1],
        y_hat_B[i-2],
        y_hat_B[i-3],
        y_hat_B[i-4],
        y_hat_B[i-5],
        cityB_x2_Predict[i],
        w_final_B2,
        b_final_B2
    )

print(y_hat_B)

[23. 24. 24. 24. 24. 24. 24. 23. 23. 23.]


In [41]:
# City C
cityC_Predict = X_Predict[:,2]
y_hat_C = np.zeros(cityC_Predict.shape[0])

y_hat_C[0] = MakeAdvancePrediction(
    cityC_Predict[0],
    cityC_Y_Valid[9],
    cityC_Y_Valid[8],
    cityC_Y_Valid[7],
    cityC_Y_Valid[6],
    cityC_Y_Valid[5],
    cityC_x2_Predict[0],
    w_final_C2,
    b_final_C2)

y_hat_C[1] = MakeAdvancePrediction(
    cityC_Predict[1],
    y_hat_C[0],
    cityC_Y_Valid[9],
    cityC_Y_Valid[8],
    cityC_Y_Valid[7],
    cityC_Y_Valid[6],
    cityC_x2_Predict[1],
    w_final_C2,
    b_final_C2)

y_hat_C[2] = MakeAdvancePrediction(
    cityC_Predict[2],
    y_hat_C[1],
    y_hat_C[0],
    cityC_Y_Valid[9],
    cityC_Y_Valid[8],
    cityC_Y_Valid[7],
    cityC_x2_Predict[2],
    w_final_C2,
    b_final_C2)

y_hat_C[3] = MakeAdvancePrediction(
    cityC_Predict[3],
    y_hat_C[2],
    y_hat_C[1],
    y_hat_C[0],
    cityC_Y_Valid[9],
    cityC_Y_Valid[8],
    cityC_x2_Predict[3],
    w_final_C2,
    b_final_C2)

y_hat_C[4] = MakeAdvancePrediction(
    cityC_Predict[4],
    y_hat_C[3],
    y_hat_C[2],
    y_hat_C[1],
    y_hat_C[0],
    cityC_Y_Valid[9],
    cityC_x2_Predict[4],
    w_final_C2,
    b_final_C2)

for i in range(5, 10):
    y_hat_C[i] = MakeAdvancePrediction(
        cityC_Predict[i],
        y_hat_C[i-1],
        y_hat_C[i-2],
        y_hat_C[i-3],
        y_hat_C[i-4],
        y_hat_C[i-5],
        cityC_x2_Predict[i],
        w_final_C2,
        b_final_C2
    )

print(y_hat_C)

[31. 30. 29. 28. 27. 25. 25. 24. 23. 23.]


### **Output Result**

In [42]:
# Output format
week = 202143
for i in range(0, 10):
    output_datalist[i][0] = week + i
    output_datalist[i][1] = y_hat_A[i]
    output_datalist[i][2] = y_hat_B[i]
    output_datalist[i][3] = y_hat_C[i]

with open(output_dataroot, 'w', newline='', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    for row in output_datalist:
        writer.writerow(row)

# **Print Coefficient**

In [43]:
# Output basic coefficient
print("Basic Model")
print("City A:", w_final_A, b_final_A)
print("City B:", w_final_B, b_final_B)
print("City C:", w_final_C, b_final_C)

print()

# Output advance coefficient
print("Advance Model")
print("City A:", w_final_A2, b_final_A2)
print("City B:", w_final_B2, b_final_B2)
print("City C:", w_final_C2, b_final_C2)

Basic Model
City A: [0.06992404 0.25826389 0.19440561 0.12419846 0.08295154 0.06698269] 4.336959501125001
City B: [0.15355932 0.32570426 0.16873347 0.09486844 0.06726585 0.0774388 ] 2.896243140671651
City C: [0.05159495 0.19798895 0.16093948 0.13484268 0.11506568 0.10414489] 4.317572656152255

Advance Model
City A: [0.07100227 0.2593663  0.19533303 0.12504718 0.08379589 0.06795366
 0.01495762] 3.8482064813760815
City B: [0.15280189 0.34026065 0.16586732 0.08673146 0.05797745 0.06938738
 0.02590842] 3.57338120213689
City C: [0.05202219 0.20655606 0.16481187 0.13654313 0.11472481 0.10294789
 0.04317371] 3.8137757726124577


# Report *(5%)*

Report should be submitted as a pdf file **hw1_report.pdf**

*   Briefly describe the difficulty you encountered 
*   Summarize your work and your reflections 
*   No more than one page






# Save the Code File
Please save your code and submit it as an ipynb file! (**hw1.ipynb**)